In [1]:
import sys,os
sys.path.append("../")

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy import stats
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
r_thrshld = 8
k_thrshld = 2

## reading the change object and clustering.

In [15]:
article_name = "Yugoslavia"
change_object_dir =  "../data/change objects/"
filename =  f"{article_name}_change.h5"

change_object_file = os.path.join(change_object_dir, filename)

In [16]:
if os.path.exists(change_object_file):
    with pd.HDFStore(change_object_file, 'r') as store:
        change_object_dataframe = store.get("data")
else:
    print("file do not exist")
change_object_dataframe.shape

(11960, 12)

## Reduce the change object.

In [17]:
change_object_dataframe["ins_length"]= change_object_dataframe["ins_tokens"].apply(lambda x: len(x))
change_object_dataframe["del_length"]= change_object_dataframe["del_tokens"].apply(lambda x: len(x))
change_object_dataframe.head(2)

ins_start_pos  \
from revision id to revision id timestamp           timegap          editor                             
298089           298090         2001-12-05 19:43:32 43 days 06:45:01 0|24.12.199.xxx 0          160.0   
                                                                                     1          162.0   

                                                                                        ins_end_pos  \
from revision id to revision id timestamp           timegap          editor                           
298089           298090         2001-12-05 19:43:32 43 days 06:45:01 0|24.12.199.xxx 0        160.0   
                                                                                     1        162.0   

                                                                                        left_neigh  \
from revision id to revision id timestamp           timegap          editor                          
298089           298090         2001-12-05 19:43:32 43 days 06:45:01 0|24.12.199.xxx 0         159   
                                                                                     1         160   

                                                                                        right_neigh  \
from revision id to revision id timestamp           timegap          editor                           
298089           298090         2001-12-05 19:43:32 43 days 06:45:01 0|24.12.199.xxx 0          160   
                                                                                     1          161   

                                                                                        del_start_pos  \
from revision id to revision id timestamp           timegap          editor                             
298089           298090         2001-12-05 19:43:32 43 days 06:45:01 0|24.12.199.xxx 0           -1.0   
                                                                                     1           -1.0   

                                                                                        del_end_pos  \
from revision id to revision id timestamp           timegap          editor                           
298089           298090         2001-12-05 19:43:32 43 days 06:45:01 0|24.12.199.xxx 0         -1.0   
                                                                                     1         -1.0   

                                                                                       ins_tokens  \
from revision id to revision id timestamp           timegap          editor                         
298089           298090         2001-12-05 19:43:32 43 days 06:45:01 0|24.12.199.xxx 0      ([[,)   
                                                                                     1      (]],)   

                                                                                       del_tokens  \
from revision id to revision id timestamp           timegap          editor                         
298089           298090         2001-12-05 19:43:32 43 days 06:45:01 0|24.12.199.xxx 0         ()   
                                                                                     1         ()   

                                                                                             left_neigh_slice  \
from revision id to revision id timestamp           timegap          editor                                     
298089           298090         2001-12-05 19:43:32 43 days 06:45:01 0|24.12.199.xxx 0  slice(129, 160, None)   
                                                                                     1  slice(130, 161, None)   

                                                                                            right_neigh_slice  \
from revision id to revision id timestamp           timegap          editor                                     
298089           298090         2001-12-05 19:43:32 43 days 06:45:01 0|24.12.199.xxx 0  slice(160, 191, None)

In [18]:
inner_join_change_object = change_object_dataframe[(change_object_dataframe["ins_tokens"]!=()) & (change_object_dataframe["del_tokens"]!=())]
display(inner_join_change_object.shape)
inner_join_change_object.head(2)

(3901, 14)

,,,,,,ins_start_pos,ins_end_pos,left_neigh,right_neigh,del_start_pos,del_end_pos,ins_tokens,del_tokens,left_neigh_slice,right_neigh_slice,left_token,right_token,ins_length,del_length
from revision id,to revision id,timestamp,timegap,editor,,,,,,,,,,,,,,,
20114,28292,2002-02-28 09:25:33,2 days 17:42:22,0|195.170.0.27,1,161.0,170.0,153,155,154.0,154.0,"(f, ., y, ., r, ., o, ., m, .)","(macedonia,)","slice(123, 154, None)","slice(155, 186, None)","(kosovo, ]], still, uncertain, ., because, the...","(,, hungarian, in, serbia, ), ,, this, led, to...",10,1
28292,34484,2002-03-12 07:33:21,11 days 22:07:48,179,2,76.0,77.0,250,252,251.0,251.0,"(warsaw, pact)","(yugoslavia,)","slice(220, 251, None)","slice(252, 283, None)","(<, /, b, >, yu, during, most, of, the, 20th, ...","(]], ., all, of, them, but, [[, serbia, ]], an...",2,1


### Removing change object with insert or delete token size more than five.

In [19]:
bykau_change_object = inner_join_change_object[~((inner_join_change_object["ins_length"] >5 ) | (inner_join_change_object["del_length"] >5) )]
bykau_change_object.shape

(2749, 14)

In [20]:
bykau_change_object.reset_index(drop=True)[["ins_tokens", "del_tokens"]].head(3)

,ins_tokens,del_tokens
0,"(warsaw, pact)","(yugoslavia,)"
1,"(,, with, some, wikification, :)","(., not, wikified, .)"
2,"(the, macedonian, republic)","(vardar, macedonia)"


### Removing low user support tokens

In [21]:
# bykau_change_object = bykau_change_object.reset_index()
bykau_change_object = bykau_change_object.groupby("ins_tokens").filter(lambda x : x.index.get_level_values("editor").nunique()>=2)
bykau_change_object = bykau_change_object.groupby("del_tokens").filter(lambda x : x.index.get_level_values("editor").nunique()>=2)
bykau_change_object.shape

(740, 14)

In [22]:
bykau_change_object.head(2)

,,,,,,ins_start_pos,ins_end_pos,left_neigh,right_neigh,del_start_pos,del_end_pos,ins_tokens,del_tokens,left_neigh_slice,right_neigh_slice,left_token,right_token,ins_length,del_length
from revision id,to revision id,timestamp,timegap,editor,,,,,,,,,,,,,,,
377336,403378,2002-10-22 18:51:34,37 days 13:31:45,4739,0,67.0,67.0,66,68,67.0,67.0,"(the,)","(a,)","slice(36, 67, None)","slice(68, 99, None)","(., png, ]], <, /, div, >, the, federal, repub...","([[, monarchy, ]], ., after, [[, world, war, i...",1,1
631448,631453,2003-01-30 16:31:50,1 days 17:53:15,0|212.59.62.221,1,489.0,489.0,485,487,486.0,486.0,"(the,)","(to,)","slice(455, 486, None)","slice(487, 518, None)","(united, nations, in, [[, 2000, ]], ., in, [[,...","(end, the, use, of, the, name, yugoslavia, in,...",1,1


In [23]:
bykau_change_object["left_string"] = bykau_change_object["left_token"].str.join(" ")
bykau_change_object["ins_string"] = bykau_change_object["ins_tokens"].str.join(" ")
bykau_change_object["del_string"] = bykau_change_object["del_tokens"].str.join(" ")
bykau_change_object["right_string"] = bykau_change_object["right_token"].str.join(" ")

### Clustering using jaccard

##### Defining jaccard similarity function.

In [24]:
edit_tokens = bykau_change_object["ins_tokens"] + bykau_change_object["del_tokens"]
left_neighbours = bykau_change_object["left_token"].apply(lambda x: x[-r_thrshld:])
right_neighbours = bykau_change_object["right_token"].apply(lambda x: x[:r_thrshld])
neighbour_tokens = left_neighbours + right_neighbours
bykau_change_object["edit_tokens"] = edit_tokens.apply(lambda x: np.unique(x))
bykau_change_object["neighbour_tokens"] = neighbour_tokens.apply(lambda x: np.unique(x))
neighbour_vec=MultiLabelBinarizer().fit_transform(bykau_change_object["neighbour_tokens"])

In [25]:
db = DBSCAN(eps=0.5, min_samples=4, metric='jaccard').fit(neighbour_vec)
bykau_change_object["neighbour_clusters"] = db.labels_

repers_first = bykau_change_object.groupby("neighbour_clusters")[["left_string","del_string", "ins_string", "right_string"]].apply(lambda x: x.style.render())
# bykau_change_object.head(2)

In [26]:
@interact( clusters_html=fixed(repers_first), group=range(bykau_change_object.groupby("neighbour_clusters").ngroups))
def display_clusters(clusters_html, group):
     return display(HTML(clusters_html.iloc[group]))

interactive(children=(Dropdown(description='group', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,…

In [27]:
clustered_bykau_change_object = bykau_change_object[bykau_change_object["neighbour_clusters"]!= -1]
group_edit_tokens = clustered_bykau_change_object.groupby("neighbour_clusters")["edit_tokens"].apply(lambda x: set(np.concatenate(x.values.ravel(), axis=0)))
context_vec = MultiLabelBinarizer().fit_transform(group_edit_tokens)
group_edit_dataframe = pd.DataFrame(group_edit_tokens.reset_index())

In [28]:
db = DBSCAN(eps=0.8, min_samples=2, metric='jaccard').fit(context_vec)
group_edit_dataframe["reclustered_group"] = db.labels_
group_edit_dataframe = group_edit_dataframe[group_edit_dataframe["reclustered_group"]!= -1]



In [29]:
final_dataframe = pd.merge( clustered_bykau_change_object, group_edit_dataframe[["reclustered_group", "neighbour_clusters"]], how="left",on="neighbour_clusters")

In [30]:
final_dataframe.index = clustered_bykau_change_object.index

#### Saving change object and its clusters

In [31]:
bykau_dir =  "../data/bykau_change_object/"
filename =  f"{article_name}_change.h5"

change_object_file = os.path.join(bykau_dir, filename)
with pd.HDFStore(change_object_file, 'w') as store:
    store.put("data", final_dataframe,)

/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['ins_tokens', 'del_tokens', 'left_neigh_slice', 'right_neigh_slice', 'left_token', 'right_token', 'left_string', 'ins_string', 'del_string', 'right_string', 'edit_tokens', 'neighbour_tokens']]

  if self.run_code(code, result):


### Grouping and showing clusters

In [19]:
final_dataframe["left_string"] = final_dataframe["left_token"].str.join(" ")
final_dataframe["ins_string"] = final_dataframe["ins_tokens"].str.join(" ")
final_dataframe["del_string"] = final_dataframe["del_tokens"].str.join(" ")
final_dataframe["right_string"] = final_dataframe["right_token"].str.join(" ")

In [20]:
final_dataframe_grouped = final_dataframe.groupby("reclustered_group")

In [21]:
repers = final_dataframe_grouped[["left_string","del_string", "ins_string", "right_string"]].apply(lambda x: x.style.render())

In [22]:
# all_html = " ".join(repers_4_full)
# file_name = article_name + "_4_full_"+str(NO_OF_CLUSTERS) + "_clusters.html"
# file_path = os.path.join("./visualisation", file_name)
# with open(file_path, 'wb') as f:
#     f.write(all_html.encode())

## Vaisualisation of clusters
##### Rerun next cell each time this notebook is reloaded, this is a bug in current jupyterlab.

In [23]:
@interact( clusters_html=fixed(repers), group=range(final_dataframe_grouped.ngroups))
def display_clusters(clusters_html, group):
     return display(HTML(clusters_html.iloc[group]))


interactive(children=(Dropdown(description='group', options=(), value=None), Output()), _dom_classes=('widget-…